In [ ]:
! pip install transformers datasets evaluate

# Fine-tune a pretrained model

In [9]:
from datasets import load_dataset

dataset = load_dataset('yelp_review_full')

In [10]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [11]:
dataset['train'][0]

{'label': 4,
 'text': "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."}

We need a tokenizer to process the text and include a padding and truncation strategy to handle any variable sequence lengths.

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-cased')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [14]:
# subset for fine-tune
small_train_dataset = tokenized_datasets['train'].shuffle(seed=101).select(range(1000))
small_eval_dataset = tokenized_datasets['test'].shuffle(seed=101).select(range(1000))

## Train

### Train with PyTorch Trainer

Huggingface Transformers provides a `Trainer` class optimized for training Transformers models, making it easier to start training without manually writing our own training loop.

In [16]:
# For the sequence classification task
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    'google-bert/bert-base-cased',
    num_labels=5,
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The warning shows that the pretrained head of the BERT model is discarded, and replaced with a randomly initialized classification head. We will fine-tune this new model head on our sequence classification task, transferring the knowledge of the pretrained model to it.

### Training hyperparameters

Next, we create a `TrainingArguments` class which contains all the hyperparameters we can tune as well as flags for activating different training options.

Specify where to save the checkpoints from our training:

In [17]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir='test_trainer')

### Evaluate

`Trainer` does not automatically evaluate model performance during training. We will need to pass `Trainer` a function to compute and report metrics. The Evaluate library provides a simple `accuracy` function we can load with the `evaluate.load` function:

In [20]:
import numpy as np
import evaluate

metric = evaluate.load('accuracy')

Before passing our predictions to `compute`, we need to convert the logits to predictions:

In [21]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    return metric.compute(predictions=predictions, references=labels)

If we would like to monitor our evaluation metrics during fine-tuning, specify the `eval_strategy` parameter in our training arguments to report the evaluation metric at the end of each epoch:

In [22]:
training_args = TrainingArguments(
    output_dir='test_trainer',
    eval_strategy='epoch',
)

### Trainer

In [23]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

## Train in native PyTorch

In [ ]:
# reset environment as needed
del model, trainer
import torch
torch.cuda.empty_cache()

Next, manually postprocess `tokenized_dataset` to prepare it for training.

1. Remove the `text` column because the model does not accept raw texts as input:

In [26]:
tokenized_datasets = tokenized_datasets.remove_columns(['text'])

2. Rename the `label` column to `labels` because the model expects the argument to be named `labels`:

In [27]:
tokenized_datasets = tokenized_datasets.rename_column('label', 'labels')

3. Set the format of the dataset to return PyTorch tensors instead of lists:

In [28]:
tokenized_datasets.set_format('torch')

Then create a smaller subset of the dataset as previously:

In [29]:
small_train_dataset = tokenized_datasets['train'].shuffle(seed=101).select(range(1000))
small_eval_dataset = tokenized_datasets['test'].shuffle(seed=101).select(range(1000))

### DataLoader

In [30]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

Load model with the number of expected labels:

In [31]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    'google-bert/bert-base-cased',
    num_labels=5,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Optimizer and learning rate scheduler

In [32]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [33]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)

lr_scheduler = get_scheduler(
    name='linear',
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [35]:
# set up device
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

### Training loop

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

### Evaluate

In [ ]:
import evaluate

metric = evaluate.load('accuracy')
model.eval()

for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch['labels'])

metric.compute()